In [1]:
import numpy as np
import copy
from tqdm import tqdm

In [2]:
# Load the graph problem from a .tsp.txt file
# data = np.loadtxt('data/eil51.tsp.txt', usecols=[1,2])
# data = np.loadtxt('test/test1.txt', usecols=[0,1])
# data

In [3]:
# Helper function to convert the coordinates into an adjacency matrix
def coordinates_to_adjacency_matrix(data,ord=2):
    a = np.zeros((len(data),len(data)))
    for i in range(len(a)):
        for j in range(len(a)):
            if not i == j:
                a[i][j] = np.linalg.norm(data[i] - data[j],ord=ord)
    return a

In [4]:
class Chromosome():
    
    # Random generated Chromosome
    #  m - number of traveling salesmans
    # def __init__(self, number_of_cities, number_of_traveling_salesman, adj = coordinates_to_adjacency_matrix(data)):
    def __init__(self, number_of_cities, number_of_traveling_salesman, adj):
        self.n = number_of_cities
        self.m = number_of_traveling_salesman
        self.adj = adj
        c = np.array(range(1,number_of_cities))
        np.random.shuffle(c)
        self.solution = np.array_split(c, self.m)
        for i in range(len(self.solution)):
            self.solution[i] = np.insert(self.solution[i],0,0)
            self.solution[i] = np.append(self.solution[i],0)
        self.fitness()
            
    # Evaluate the Chromosome - Fitness function
    #  based on 2 features: 
    #   - overall cost (cumulated from all salesman)
    #   - worst (longest) salesman cost
    #  adj - adjacency matrix
    def fitness(self):
        self.cost = 0
        self.minmax=0
        # longest_salesman_fitness = []
        # longest_salesman_length = 0
        for i in range(self.m):
            salesman = self.solution[i]
            salesman_fitness = 0
            for j in range(len(salesman) - 1):
                salesman_fitness = salesman_fitness + self.adj[salesman[j]][salesman[j+1]]
            self.cost = self.cost + salesman_fitness
            # if len(salesman) > longest_salesman_length or (len(salesman) == longest_salesman_length and salesman_fitness > self.minmax):
            #     longest_salesman_length = len(salesman)
            #     self.minmax = salesman_fitness
            # if len(salesman) > longest_salesman_length:
            #     longest_salesman_length = len(salesman)
            if salesman_fitness > self.minmax:
                self.minmax = salesman_fitness
        self.score =  self.minmax
        # self.print()

    # Mutation operator - mutates a single Traveling Salesman
    #  by swaping 2 cities
    def mutate_local(self):
        index = np.random.randint(0,self.m)
        mutant = self.solution[index]
        i,j = np.random.randint(1,len(mutant)-1), np.random.randint(1,len(mutant)-1)
        mutant[i], mutant[j] = mutant[j], mutant[i]
        old_cost = self.cost
        self.fitness()
    
    # Mutation operator - mutates 2 Traveling Salesmans
    #  by removing a city from a salesman and asigning it to the second one
    def mutate_global(self):
        for i in range(self.m):
            if len(self.solution[i]) < 3:
                print(i, self.solution[i])
        
        
        index1, index2 = np.random.randint(0,self.m), np.random.randint(0,self.m)
        while index1 == index2:
            index1, index2 = np.random.randint(0,self.m), np.random.randint(0,self.m)
        while len(self.solution[index1]) < 4:
            index1, index2 = np.random.randint(0,self.m), np.random.randint(0,self.m)
        mutant1, mutant2 = self.solution[index1], self.solution[index2]
        i,j = np.random.randint(1,len(mutant1)-1), np.random.randint(1,len(mutant2)-1)
        self.solution[index2] = np.insert(mutant2, j, mutant1[i])
        self.solution[index1] = np.delete(mutant1, i)
        old_cost = self.cost
        self.fitness()

    def print(self):
        total_cost = 0
        minmax = 0
        for i in range(self.m):
            salesman = self.solution[i]
            cost=0
            print(i+1, ":  ", self.solution[i][0]+1, end="", sep="")
            for j in range(1,len(self.solution[i])):
                # print("-", self.solution[i][j]+1, end="", sep="")
                dist=self.adj[salesman[j-1]][salesman[j]]
                print("[%.0f]%d"%(dist,self.solution[i][j]+1), end="", sep="")
                cost+=dist
            total_cost+=cost
            if cost>minmax:
                minmax = cost
            print(" --- %.0f#"%(cost), len(self.solution[i]))
        # print("Cost:   \t%.1f\t%.1f"%(self.cost,total_cost))
        # print("Minmax: \t%.1f\t%.1f"%(self.minmax,minmax))
        print("Cost:   \t%.1f"%(total_cost))
        print("Minmax: \t%.1f"%(minmax))
    

In [5]:
def optimize(n_of_ts,coordinates,order,cycle=100000):
    adjacency = coordinates_to_adjacency_matrix(coordinates,ord=order)
    n_cities = len(coordinates)
    # print("n_cities:",n_cities)
    chromosome = Chromosome(number_of_cities = n_cities, number_of_traveling_salesman = n_of_ts, adj=adjacency)
    for it in tqdm(range(cycle)):
        # Mutate globally
        chromosome_copy = copy.deepcopy(chromosome)
        chromosome_copy.mutate_global()
        if chromosome_copy.score < chromosome.score:
            chromosome = chromosome_copy
        # Mutate locally
        chromosome_copy = copy.deepcopy(chromosome)
        chromosome_copy.mutate_local()
        if chromosome_copy.score < chromosome.score:
            chromosome = chromosome_copy
    return(chromosome)

In [9]:
# data = np.loadtxt('data/eil51.tsp.txt', usecols=[1,2])
data = np.loadtxt('test/test1.txt', usecols=[0,1])
data

ch=optimize(n_of_ts=2,coordinates=data,order=1,cycle=100000)

100%|██████████| 10000/10000 [00:18<00:00, 542.84it/s]


In [10]:
ch.print()

1:  1[1204]319[1067]60[398]286[1230]255[1565]337[1173]138[1935]402[704]40[2355]356[2123]80[558]62[428]500[1583]368[192]279[1364]16[163]12[657]48[280]474[1436]206[1032]208[1801]81[630]19[964]59[1487]429[787]26[687]76[1164]493[414]161[153]43[2229]252[299]367[22]475[449]328[1278]34[387]125[1662]242[1281]38[939]270[231]85[939]141[409]378[604]446[618]183[323]453[2169]215[1281]129[1289]107[2247]180[415]199[2209]65[1728]89[1738]136[562]267[975]194[1175]13[1337]372[569]415[1683]245[195]115[792]236[387]272[320]99[1821]443[1902]164[1051]361[1354]36[1349]419[1430]172[2229]174[160]283[1017]17[1213]301[1794]275[313]173[33]154[2498]159[1108]497[346]381[953]234[157]188[1168]360[418]121[965]114[293]244[156]273[830]186[544]282[759]348[406]382[964]212[549]168[853]370[991]95[483]24[614]196[1750]399[2311]181[669]458[510]224[1141]47[1039]320[743]469[1542]457[1618]198[718]397[660]175[455]100[196]274[1256]42[1208]63[277]7[1450]418[1733]291[1536]292[461]391[1878]73[1282]313[251]394[443]230[1885]130[1145]331[6